In [24]:
import pandas as pd

# Cargar el archivo de datos
file_path = '/Users/Abraham/iCloud Drive (Archive)/Documents/ML-Python/SQL_TEST.xlsx'
with pd.ExcelFile(file_path, engine='openpyxl') as xls:
    data = pd.read_excel(xls, header=2)

# Dividir el DataFrame en tres DataFrames diferentes y renombrar columnas
Customer = data[['Customerid', 'Email', 'Name', 'LastName', 'Gender']]
Customer.columns = ['Customerid', 'Email', 'Name', 'LastName', 'Gender']

Product = data[['ProductID', 'Name.1', 'Datesale', 'Channel', 'Currency code', 'Amount', 'Customerid.1', 'Stationid']]
Product.columns = ['ProductID', 'ProductName', 'Datesale', 'Channel', 'CurrencyCode', 'Amount', 'Customerid', 'Stationid']

Station = data[['Stationid.1', 'Region', 'City']]
Station.columns = ['Stationid', 'Region', 'City']

# Consulta 1: Clientes con más compras en cada región MX y USA
try:
    merged_df = pd.merge(pd.merge(Customer, Product, on='Customerid'), Station, on='Stationid')
    filtered_df = merged_df[merged_df['Region'].isin(['MX', 'USA'])]

    top_customers = filtered_df.groupby(['Region', 'Name', 'LastName'], as_index=False)['Amount'].sum()
    top_customers = top_customers.loc[top_customers.groupby('Region')['Amount'].idxmax()]

    print("Consulta 1:")
    print(top_customers[['Region', 'Name', 'LastName']])
    print()
except KeyError as e:
    print(f"Error en la consulta 1: {e}")
    print("Columnas disponibles en merged_df:", merged_df.columns.tolist())

# Consulta 2: Emails de clientes mujeres con valor de productos comprados mayor a $100
try:
    female_customers = Customer[Customer['Gender'] == 1]
    high_value_purchases = Product[Product['Amount'] > 100]

    female_high_value_emails = pd.merge(female_customers, high_value_purchases, on='Customerid')['Email'].unique()

    print("Consulta 2:")
    print(female_high_value_emails)
    print()
except KeyError as e:
    print(f"Error en la consulta 2: {e}")
    print("Columnas disponibles en female_customers:", female_customers.columns.tolist())
    print("Columnas disponibles en high_value_purchases:", high_value_purchases.columns.tolist())

# Consulta 3: Número de productos, número de clientes y amount total por región
try:
    summary_df = merged_df.groupby('Region').agg(
        NumProducts=pd.NamedAgg(column='ProductID', aggfunc='nunique'),
        NumCustomers=pd.NamedAgg(column='Customerid', aggfunc='nunique'),
        TotalAmount=pd.NamedAgg(column='Amount', aggfunc='sum')
    ).reset_index()

    print("Consulta 3:")
    print(summary_df)
except KeyError as e:
    print(f"Error en la consulta 3: {e}")
    print("Columnas disponibles en merged_df:", merged_df.columns.tolist())


Consulta 1:
  Region      Name        LastName
1     MX  ANAPAOLA  MUNOZ ARELLANO

Consulta 2:
['email2' 'email3' 'email5' 'email9' 'email10']

Consulta 3:
  Region  NumProducts  NumCustomers  TotalAmount
0     MX            7             7         4170
1     US            6             6          252
